In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r drive/MyDrive/diploma/deepset/* .

In [ ]:
!cp drive/MyDrive/diploma/deepset/train_meta_tensors.npy .

In [ ]:
!cp drive/MyDrive/diploma/deepset/test_meta_tensors.npy .

In [ ]:
!unzip processed_data.zip   

In [ ]:
class CONFIG:
    # DATA
    features = 16
    instances = 64
    classes = 2
    z_length = 128

    # TRAIN
    num_epochs = 20
    train_batch_size = 1024
    learning_rate = 0.002
    criterion = 'torch.nn.BCEWithLogitsLoss'

    test_batch_size = 1024

    # MODEL
    hidden_size_0=128
    hidden_size_1=256
    predlast_hidden_size=512
    meta_size=27
    out_classes=4
    standardscaler=True
    meta_standardscaler=True
    transpose=True

    # device = torch.device("cuda:0" if torch.cuda.is_available() else if torch.backends.mps.is_available() then "mps" else "cpu")
    # device is cuda is available else mps is available else cpu
    device_mps_or_cpu = ("mps" if torch.backends.mps.is_available() else "cpu")
    device = torch.device("cuda:0" if torch.cuda.is_available() else device_mps_or_cpu)

    # OTHER
    seed = 42
    num_workers = 8


## Model

### Discriminator = model v4

In [ ]:
from deepsets import deepsetlayers
import torch.nn.functional as F

class Discriminator(torch.nn.Module):
    '''
    DeepSetModelV3 discriminator
    '''
    def __init__(self, hidden_size_0=1, hidden_size_1=1, predlast_hidden_size=1, meta_size = 27, out_classes=1):
        super().__init__()
        self.hidden_size_0 = hidden_size_0
        self.hidden_size_1 = hidden_size_1
        self.predlast_hidden_size = predlast_hidden_size
        self.meta_size = meta_size
        self.out_classes = out_classes
        self.inv_0 = deepsetlayers.InvLinear(hidden_size_0, hidden_size_0)
        self.inv_1 = deepsetlayers.InvLinear(2 * hidden_size_1, 4 * hidden_size_1)
        self.equiv_0 = deepsetlayers.EquivLinear(1, hidden_size_0)
        self.equiv_1 = deepsetlayers.EquivLinear(hidden_size_0, 2 * hidden_size_1)
        self.relu = torch.nn.ReLU()
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(4 * hidden_size_1 + meta_size, 2 * predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(2 * predlast_hidden_size, predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(predlast_hidden_size, out_classes),
        )

    def forward(self, x, y):
        x = x.unsqueeze(-1)
        # (batch_size, N, M, 1)
        N = x.shape[1]
        x = x.flatten(0, 1)
        # (batch_size * N, M, 1)
        x = self.equiv_0(x)
        # (batch_size * N, M, hidden_size_0)
        x = self.relu(x)
        x = self.inv_0(x)
        # (batch_size * N, hidden_size_0)
        x = x.reshape(-1, N, self.hidden_size_0)
        # (batch_size, N, hidden_size_0)

        x = self.equiv_1(x)
        # (batch_size, N, hidden_size_1)
        x = self.relu(x)
        x = self.inv_1(x)
        # (batch_size, hidden_size_1)

        y = y.view(-1, self.meta_size)
        # (batch_size, meta_size)        

        x = torch.hstack([x, y])
        # (batch_size, hidden_size_1 + meta_size)
        x = self.regressor(x)
        # (batch_size, out_classes)
        return x

### Generator

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
from torch import nn

class Generator(torch.nn.Module):
    def __init__(self, features_size, instances_size, classes_size, meta_length, z_length):
        super(Generator, self).__init__()
        
        # nz = # Size of z latent vector (i.e. size of generator input)
        # ngf = # Size of feature maps in discriminator
        # nc = # Number of channels in the training samples. For dataset this is 1,
        # but for torch.cat([dataset, target_map], dim=2) this is 2
        # self.nc = nc
        # self.nz = nz
        # self.nfg = nfg

        self.features_size = features_size
        self.instances_size = instances_size
        self.data_size = instances_size
        self.classes_size = classes_size
        self.meta_length = meta_length
        self.z_length = z_length


        # self.fc_z = nn.ConvTranspose2d(in_channels=self.z_length,
        #                                out_channels=self.data_size * 4, kernel_size=4, stride=1, padding=0)
        # # in (?, meta_length, 1, 1)
        # # out (?, data_size * 4, 4, 4)
        # self.fc_meta = nn.ConvTranspose2d(in_channels=self.meta_length,
        #                                   out_channels=self.data_size * 4, kernel_size=4, stride=1, padding=0)
        # # in (?, data_size * 8, 4, 4)
        # # out (?, data_size * 4, 8, 8)
        # self.deconv1 = nn.ConvTranspose2d(in_channels=self.data_size * 8,
        #                                   out_channels=self.data_size * 4, kernel_size=4, stride=2, padding=1)
        # # out (?, data_size * 2, 16, 16)
        # self.deconv2 = nn.ConvTranspose2d(in_channels=self.data_size * 4,
        #                                   out_channels=self.data_size * 2, kernel_size=4, stride=2, padding=1)
        # # out (?, data_size, 32, 16)
        # self.deconv3 = nn.ConvTranspose2d(in_channels=self.data_size * 2,
        #                                   out_channels=self.data_size, kernel_size=(4, 1), stride=(2, 1),
        #                                   padding=(1, 0))
        # # out (?, data_size / 2, 64, 16)
        # self.deconv4 = nn.ConvTranspose2d(in_channels=self.data_size,
        #                                   out_channels=classes_size, kernel_size=(4, 1), stride=(2, 1), padding=(1, 0))

        self.upconv_z = nn.ConvTranspose2d(in_channels=self.z_length,
                                       out_channels=self.data_size * 4, kernel_size=4, stride=1, padding=0)
        self.upconv_meta = nn.ConvTranspose2d(in_channels=self.meta_length,
                                          out_channels=self.data_size * 4, kernel_size=4, stride=1, padding=0)
        
        # after cat:
        # (?, data_size * 8, 4, 4)

        self.main_upconv = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=8 * self.data_size,
                                out_channels=self.data_size * 4, kernel_size=4, 
                                stride=2, padding=1, bias=False),
            # nn.BatchNorm(self.data_size * 8),
            nn.ReLU(True),
            ### ^ - (?, data_size * 8, 8, 8)
            nn.ConvTranspose2d(in_channels=self.data_size * 4,
                               out_channels=self.data_size * 2, kernel_size=4, 
                               stride=2, padding=1, bias=False),
            # nn.BatchNorm(self.data_size * 4),
            nn.ReLU(True),
            ### ^ - (?, data_size * 4, 16, 16)
            nn.ConvTranspose2d(in_channels=self.data_size * 2,
                               out_channels=self.data_size, kernel_size=(4, 1), 
                               stride=(2, 1), padding=(1, 0)),
            # nn.BatchNorm(self.data_size * 2),
            nn.ReLU(True),
            ### ^ - (?, data_size, 32, 16)
            nn.ConvTranspose2d(in_channels=self.data_size,
                                          out_channels=classes_size, kernel_size=(4, 1), stride=(2, 1), padding=(1, 0)),
            nn.Tanh()
        )

    def forward(self, z, meta):
        upsampled_z = self.upconv_z(z)
        upsampled_meta = self.upconv_meta(meta)

        upsampled_conditional_vector = torch.cat((upsampled_z, upsampled_meta), 1)

        result = self.main_upconv(upsampled_conditional_vector)
        return result


In [ ]:
d_model = Discriminator(hidden_size_0=CONFIG.hidden_size_0, 
                     hidden_size_1=CONFIG.hidden_size_1, 
                     predlast_hidden_size=CONFIG.predlast_hidden_size, 
                     meta_size=CONFIG.meta_size,
                     out_classes=CONFIG.out_classes)

g_model = Generator(CONFIG.features,
                    CONFIG.instances,
                    CONFIG.classes,
                    CONFIG.meta_size,
                    CONFIG.z_length)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
g_out = g_model(torch.randn(CONFIG.train_batch_size, CONFIG.z_length).unsqueeze(-1).unsqueeze(-1), 
                b.unsqueeze(-1).unsqueeze(-1))
g_out.shape

torch.Size([1024, 2, 64, 16])

In [ ]:
{'generator_params': count_parameters(g_model),
 'discriminator_params': count_parameters(d_model)}

{'generator_params': 3290178, 'discriminator_params': 2277892}

## Data

In [ ]:
import os

import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector


class LambdaDatasetFolder(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 lambdas: LambdaFeaturesCollector):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.lambda_features = lambdas

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        lambda_tensor = self.lambda_features.get(data_np)
        return dataset_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)

In [ ]:
lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

train_dataset = LambdaDatasetFolder('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              lambdaFeaturesCollector)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = LambdaDatasetFolder('processed_data/test/',
                                CONFIG.features,
                                CONFIG.instances,
                                CONFIG.classes,
                                test_lambdaFeaturesCollector)

shapes = Counter({(2, 64, 16): 8000})
shapes = Counter({(2, 64, 16): 1911})


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

## DataSet With Metas

In [ ]:
import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector
from sklearn.preprocessing import StandardScaler
import os

class DatasetWithMetaFolder(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 meta: MetaFeaturesCollector, lambdas: LambdaFeaturesCollector, train_meta: bool,
                 meta_precalc_path: str, standardscale=False):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        self.meta_precalc_path = meta_precalc_path
        self.meta_precalculated = np.load(self.meta_precalc_path, allow_pickle=True)
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                # exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.meta_features = meta
        if train_meta:
            self.meta_features.train(self.root, load_from_fs=True)
        
        self.standardscale = standardscale
        self.lambda_features = lambdas
        self.data_scaler = StandardScaler()
        self.all_data = np.concatenate([np.load(i, allow_pickle=True) for i in self.data_paths])
        self.all_data = self.all_data.reshape((-1, self.all_data.shape[-1]))
        print(f"shape = {self.all_data.shape}")
        self.data_scaler.fit(self.all_data)
        del self.all_data

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        # meta_tensor = self.meta_features.get(data_np)
        meta_tensor = torch.Tensor(self.meta_precalculated[index])
        meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
        lambda_tensor = self.lambda_features.get(data_np)

        if self.standardscale:
            dataset_tensor = torch.Tensor(self.data_scaler.transform(dataset_tensor))

        return dataset_tensor, meta_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)


In [ ]:
lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)
metaFeaturesCollector = MetaFeaturesCollector(CONFIG.features, CONFIG.instances)
train_dataset = DatasetWithMetaFolder('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              True,
                              'train_meta_tensors.npy',
                              standardscale=CONFIG.standardscaler)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = DatasetWithMetaFolder('processed_data/test/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              False,
                              'test_meta_tensors.npy',
                              standardscale=CONFIG.standardscaler)

shapes = Counter({(2, 64, 16): 8000})
processed_data/processed_16_64_2/
Loaded from file system
shape = (1024000, 16)
shapes = Counter({(2, 64, 16): 1911})
shape = (244608, 16)


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

## dataset with cached lambdas and norm



In [ ]:
import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector
from sklearn.preprocessing import StandardScaler
import os

class DatasetWithMetaFolder(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 meta: MetaFeaturesCollector, lambdas: LambdaFeaturesCollector, train_meta: bool,
                 meta_precalc_path: str = None, lambda_precalc_path: str = None, 
                 standardscale=False, meta_standardscale=False, transpose=False):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        self.metas = meta
        self.lambdas = lambdas
        self.meta_precalc_path = meta_precalc_path
        self.lambda_precalc_path = lambda_precalc_path
        if self.meta_precalc_path is not None:
            self.meta_precalculated = np.load(self.meta_precalc_path, allow_pickle=True)
        if self.lambda_precalc_path is not None:
            self.lambda_precalculated = np.load(self.lambda_precalc_path, allow_pickle=True)
        self.standardscale = standardscale
        self.meta_standardscale = meta_standardscale
        self.transpose = transpose
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                # exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.meta_features = meta
        if train_meta:
            self.meta_features.train(self.root, load_from_fs=True)
        
        self.lambda_features = lambdas
        # standardscale
        if self.standardscale:
            self.data_scaler = StandardScaler()
            self.all_data = np.concatenate([np.load(i, allow_pickle=True) for i in self.data_paths])
            self.all_data = self.all_data.reshape((-1, self.all_data.shape[-1]))
            print(f"shape = {self.all_data.shape}")
            self.data_scaler.fit(self.all_data)
            del self.all_data

        # meta_standard_scale

        if self.meta_standardscale:
            self.meta_data_scaler = StandardScaler()
            all_meta_data = None
            if self.meta_precalc_path is not None:
                all_meta_data = self.meta_precalculated
            else:
                raise NotImplementedError(":)")
            self.meta_data_scaler.fit(all_meta_data.reshape(-1, self.meta_features.getLength()))

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        # meta_tensor = self.meta_features.get(data_np)
        meta_tensor = torch.Tensor(self.meta_precalculated[index])
        meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
        lambda_tensor = torch.Tensor(self.lambda_precalculated[index])

        if self.standardscale:
            dataset_tensor = torch.Tensor(self.data_scaler.transform(dataset_tensor))

        if self.meta_standardscale:
            meta_tensor = torch.Tensor(self.meta_data_scaler.transform(meta_tensor.reshape(-1, self.meta_features.getLength()))).reshape(-1)

        total_elems = self.classes * self.instances
        dataset_tensor = torch.cat([dataset_tensor, 
                   (torch.arange(0, total_elems) < total_elems / 2).reshape((total_elems, 1))],
                #    (torch.arange(0, total_elems) >= total_elems / 2).reshape((total_elems, 1))], 
                  dim=1)
        if self.transpose:
            dataset_tensor = dataset_tensor.T
        return dataset_tensor, meta_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)

    def precalc(self, pref_name):
        from tqdm import tqdm
        meta_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            meta_tensor = self.meta_features.get(data_np)
            meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
            meta_tensors.append(meta_tensor)
        
        # write meta_tensors to numpy_file
        meta_tensors = torch.stack(meta_tensors).numpy()
        np.save(f'{pref_name}_meta_tensors.npy', meta_tensors)

    def precalc_lambda(self, pref_name):
        from tqdm import tqdm
        lambda_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            lambda_tensor = self.lambda_features.get(data_np)
            lambda_tensors.append(lambda_tensor)
            
        # write meta_tensors to numpy_file
        lambda_tensors = torch.stack(lambda_tensors).numpy()
        print(lambda_tensors.shape)
        np.save(f'{pref_name}_lambda_tensors.npy', lambda_tensors)


from copy import deepcopy

lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)
metaFeaturesCollector = MetaFeaturesCollector(CONFIG.features, CONFIG.instances)
train_dataset = DatasetWithMetaFolder('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              True,
                              meta_precalc_path='train_meta_tensors.npy',
                              lambda_precalc_path='train_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = DatasetWithMetaFolder('processed_data/test/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              False,
                              meta_precalc_path='test_meta_tensors.npy',
                              lambda_precalc_path='test_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_dataset.data_scaler = deepcopy(train_dataset.data_scaler)
test_dataset.meta_data_scaler = deepcopy(test_dataset.meta_data_scaler)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

shapes = Counter({(2, 64, 16): 8000})
processed_data/processed_16_64_2/
Loaded from file system
shape = (1024000, 16)
shapes = Counter({(2, 64, 16): 1911})
shape = (244608, 16)


## dataset with labels



In [ ]:
import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector
from sklearn.preprocessing import StandardScaler
import os

class DatasetWithTargets(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 meta: MetaFeaturesCollector, lambdas: LambdaFeaturesCollector, train_meta: bool,
                 meta_precalc_path: str = None, lambda_precalc_path: str = None, 
                 standardscale=False, meta_standardscale=False, transpose=False):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        self.metas = meta
        self.lambdas = lambdas
        self.meta_precalc_path = meta_precalc_path
        self.lambda_precalc_path = lambda_precalc_path
        if self.meta_precalc_path is not None:
            self.meta_precalculated = np.load(self.meta_precalc_path, allow_pickle=True)
        if self.lambda_precalc_path is not None:
            self.lambda_precalculated = np.load(self.lambda_precalc_path, allow_pickle=True)
        self.standardscale = standardscale
        self.meta_standardscale = meta_standardscale
        self.transpose = transpose
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                # exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.meta_features = meta
        if train_meta:
            self.meta_features.train(self.root, load_from_fs=True)
        
        self.lambda_features = lambdas
        # standardscale
        if self.standardscale:
            self.data_scaler = StandardScaler()
            self.all_data = np.concatenate([np.load(i, allow_pickle=True) for i in self.data_paths])
            self.all_data = self.all_data.reshape((-1, self.all_data.shape[-1]))
            print(f"shape = {self.all_data.shape}")
            self.data_scaler.fit(self.all_data)
            del self.all_data

        # meta_standard_scale

        if self.meta_standardscale:
            self.meta_data_scaler = StandardScaler()
            all_meta_data = None
            if self.meta_precalc_path is not None:
                all_meta_data = self.meta_precalculated
            else:
                raise NotImplementedError(":)")
            self.meta_data_scaler.fit(all_meta_data.reshape(-1, self.meta_features.getLength()))

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        # meta_tensor = self.meta_features.get(data_np)
        meta_tensor = torch.Tensor(self.meta_precalculated[index])
        meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
        lambda_tensor = torch.Tensor(self.lambda_precalculated[index])

        if self.standardscale:
            dataset_tensor = torch.Tensor(self.data_scaler.transform(dataset_tensor))

        if self.meta_standardscale:
            meta_tensor = torch.Tensor(self.meta_data_scaler.transform(meta_tensor.reshape(-1, self.meta_features.getLength()))).reshape(-1)

        total_elems = self.classes * self.instances
        labels = (torch.arange(0, total_elems) < total_elems / 2).reshape(total_elems)
                   
    
        dataset_tensor = torch.cat([dataset_tensor, 
                   (torch.arange(0, total_elems) < total_elems / 2).reshape((total_elems, 1)), 
                   (torch.arange(0, total_elems) >= total_elems / 2).reshape((total_elems, 1))], 
                  dim=1)
        
        pos_ = dataset_tensor[labels]
        neg_ = dataset_tensor[~labels]
        
        if self.transpose:
            dataset_tensor = dataset_tensor.T
            pos_ = pos_.T
            neg_ = neg_.T
        return pos_, neg_, meta_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)

    def precalc(self, pref_name):
        from tqdm import tqdm
        meta_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            meta_tensor = self.meta_features.get(data_np)
            meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
            meta_tensors.append(meta_tensor)
        
        # write meta_tensors to numpy_file
        meta_tensors = torch.stack(meta_tensors).numpy()
        np.save(f'{pref_name}_meta_tensors.npy', meta_tensors)

    def precalc_lambda(self, pref_name):
        from tqdm import tqdm
        lambda_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            lambda_tensor = self.lambda_features.get(data_np)
            lambda_tensors.append(lambda_tensor)
            
        # write meta_tensors to numpy_file
        lambda_tensors = torch.stack(lambda_tensors).numpy()
        print(lambda_tensors.shape)
        np.save(f'{pref_name}_lambda_tensors.npy', lambda_tensors)


from copy import deepcopy

lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)
metaFeaturesCollector = MetaFeaturesCollector(CONFIG.features, CONFIG.instances)
train_dataset = DatasetWithTargets('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              True,
                              meta_precalc_path='train_meta_tensors.npy',
                              lambda_precalc_path='train_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = DatasetWithTargets('processed_data/test/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              False,
                              meta_precalc_path='test_meta_tensors.npy',
                              lambda_precalc_path='test_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_dataset.data_scaler = deepcopy(train_dataset.data_scaler)
test_dataset.meta_data_scaler = deepcopy(test_dataset.meta_data_scaler)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

shapes = Counter({(2, 64, 16): 8000})
processed_data/processed_16_64_2/
Loaded from file system
shape = (1024000, 16)
shapes = Counter({(2, 64, 16): 1911})
shape = (244608, 16)


## Init Model + Optimizer + Criterion

In [ ]:
d_model = Discriminator(hidden_size_0=CONFIG.hidden_size_0, 
                     hidden_size_1=CONFIG.hidden_size_1, 
                     predlast_hidden_size=CONFIG.predlast_hidden_size, 
                     meta_size=CONFIG.meta_size,
                     out_classes=CONFIG.out_classes)

g_model = Generator(CONFIG.features,
                    CONFIG.instances,
                    CONFIG.classes,
                    CONFIG.meta_size,
                    CONFIG.z_length)

g_model = g_model.apply(weights_init)

d_model = d_model.to(CONFIG.device)
g_model = g_model.to(CONFIG.device)

d_optimizer = torch.optim.Adam(d_model.parameters(), lr=CONFIG.learning_rate)
g_optimizer = torch.optim.Adam(g_model.parameters(), lr=CONFIG.learning_rate)
# criterion = torch.nn.MSELoss()
BCE = torch.nn.BCEWithLogitsLoss()
MSE = torch.nn.MSELoss()

In [ ]:
 !pip install wandb==0.12.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=28eabd9800541b97330ae85f499ccf844479e83e0bd269b7aee3b8174712fb6f
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
# init wandb

import wandb

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

cfg = dict(CONFIG.__dict__)
cfg.pop('__module__')
cfg.pop('__dict__')
cfg.pop('__weakref__')
cfg.pop('__doc__')
cfg['generator_parameteres_num'] = count_parameters(g_model)
cfg['discriminator_parameters_num'] = count_parameters(d_model)

print(cfg)

run = wandb.init(project="diploma1234", config=cfg, id='gan', notes='first gan attempt, discriminator is deepmodelV3, generator like in LM-DCGAN, but instead of mse for probs -- BCEWithLogitsLoss')

run.watch(g_model)
run.watch(d_model)

ModuleNotFoundError: ignored

## train

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def getMeta(data_in: torch.Tensor):
    meta_list = []
    for data in data_in:
        meta_list.append(train_dataset.metas.getShort(data.cpu().detach().numpy()))
    result = torch.stack(meta_list)
    return result.view((result.size(0), result.size(1), 1, 1)).to(CONFIG.device)

def getLambda(self, data_in: torch.Tensor):
    lamba_list = []
    for data in data_in:
        lamba_list.append(train_dataset.lambdas.get(data.cpu().detach().numpy()))
    result = torch.stack(lamba_list)
    return result.to(CONFIG.device)


In [ ]:
def transform_flatten_and_cat(dataset_tensor):
    total_elems = CONFIG.instances * CONFIG.classes
    dataset_tensor = fake_data.flatten(1, 2)
    result = torch.cat([dataset_tensor, 
                   (torch.arange(0, total_elems) < total_elems / 2).reshape((total_elems, 1))],
          dim=1)

    return result

In [ ]:
for epoch in range(CONFIG.num_epochs):
    # train loop
    train_g_epoch_loss = 0
    train_d_epoch_loss = 0
    for (ind, (X, meta, y)) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Train Epoch {epoch}"):
        X = X.to(CONFIG.device)
        meta = meta.to(CONFIG.device)
        unsqueezed_meta = meta.unsqueeze(-1).unsqueeze(-1)
        y = y.to(CONFIG.device)
        noise = torch.randn(CONFIG.train_batch_size, CONFIG.z_length) \
            .unsqueeze(-1).unsqueeze(-1).to(CONFIG.device)
        zeros = torch.zeros([CONFIG.train_batch_size, 1], dtype=torch.float32, 
                            device=CONFIG.device)
        ones = torch.ones([CONFIG.train_batch_size, 1], dtype=torch.float32, 
                            device=CONFIG.device)
        
        # Get D on real
        real_outputs = d_model(X, meta)
        d_real_labels_loss = MSE(real_outputs[:, 1:], y)
        d_real_rf_loss = MSE(real_outputs[:, :1], zeros) # :TODO: CHANGE TO BCE
        d_real_loss = d_real_labels_loss + 0.7 * d_real_rf_loss

        # Get D on fake
        fake_data = g_model(noise, unsqueezed_meta)
        fake_data_metas = getMeta(fake_data)
        fake_outputs = d_model(fake_data, meta)
        fake_lambdas = getLambda(fake_data)

        smoothing_coef = torch.exp(-torch.square(torch.norm(unsqueezed_meta - fake_data_metas)))
        d_fake_labels_loss = MSE(fake_outputs[:, 1:], fake_lambdas)
        d_fake_rf_loss = MSE(fake_outputs[:, :1], ones) # :TODO: CHANGE TO BCE
        d_fake_loss = 0.7 * d_fake_rf_loss + smoothing_coef * d_fake_labels_loss

        # Train D
        d_loss = d_real_loss + 0.8 * d_fake_loss
        g_model.zero_grad()
        d_model.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        train_d_epoch_loss += d_loss

        # Get D on fake
        noise = torch.randn(CONFIG.train_batch_size, CONFIG.z_length) \
            .unsqueeze(-1).unsqueeze(-1).to(CONFIG.device)
        fake_data = g_model(noise, unsqueezed_meta)
        fake_outputs = d_model(fake_data, meta)
        g_fake_rf_loss = MSE(fake_outputs[:, :1], zeros) # :TODO: CHANGE TO BCE
        fake_metas = getMeta(fake_data)
        g_fake_meta_loss = MSE(fake_metas, unsqueezed_meta)
        g_loss = 0.7 * g_fake_rf_loss + g_fake_meta_loss

        # Train G
        g_model.zero_grad()
        d_model.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        train_g_epoch_loss += g_loss.item()

    train_g_epoch_loss = train_g_epoch_loss / len(train_dataloader)
    train_d_epoch_loss = train_d_epoch_loss / len(train_dataloader)

    # test loop
    test_epoch_loss = 0
    test_squared_error = 0
    test_total_elems = 0
    for (ind, (X, meta, y)) in tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc=f"Test Epoch {epoch}"):
        X = X.to(CONFIG.device)
        meta = meta.to(CONFIG.device)
        y = y.to(CONFIG.device)

        y_pred = d_model(X, meta)
        loss = MSE(y_pred, y)

        test_epoch_loss += loss.item()

        # y_pred_ = torch.sigmoid(y_pred)
        y_pred_ = y_pred[:, 1:]

        # update test squared error without mean
        test_squared_error += torch.sum((y_pred_ - y) ** 2).item()
        test_total_elems += y_pred.shape[0] * y_pred.shape[1]


    test_epoch_loss = test_epoch_loss / len(test_dataloader)
    test_mse = test_squared_error / test_total_elems
    wandb.log({"train_g_loss": train_g_epoch_loss,
               "train_d_loss": train_d_epoch_loss,
               "test_loss": test_epoch_loss,
               "test_mse": test_mse})



    print(f'Epoch {epoch} train generator loss: {train_g_epoch_loss}')
    print(f'Epoch {epoch} train discriminator loss: {train_d_epoch_loss}')
    print(f'Epoch {epoch} test loss: {test_epoch_loss}')
    print(f'Epoch {epoch} test mse: {test_mse}')
    print()

wandb.finish()

Train Epoch 0:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: ignored